# About this notebook
this notebook is written by changhyun yi, taking Modulab's Aiffel Program (Deep learning education course).  
이 노트북은 모두의연구소 AIFFEL 과정 중 going-deeper node 16, hugging face custom model 만들기에 제출하기 위해 만들었습니다.

# mnli task
MNLI task에 대한 description은 [MultiNLI](https://cims.nyu.edu/~sbowman/multinli/) 에서 찾을 수 있습니다. 두 문장이 __전제(premise)__ 와 __가설(hypothesis)__ 로 제시되고 그 사이의 관계를 entailment - 이어짐, contradict - 반(反)함, neutral - 관계없음 세 label로 나눕니다. 

node에서 다룬 것은 정제된 BERT pretrain모델입니다. 이것은 label없이 말뭉치에서 마스크를 씌우고 문장이 이어지는지를 라벨로 하여 학습시킨 것이므로, 두 문장의 관계를 따지는 것이라 연관이 아예 없어보이지는 않는데 안된다니 아쉽네요.  


# MNLI dataset 둘러보기

tensorflow-datasets 패키지와 포함된 datasets에 리스트에 대해서는 https://www.tensorflow.org/datasets/catalog/overview#all_datasets, 
그 중 GLUE에 관한 부분은 https://www.tensorflow.org/datasets/catalog/glue 에서 볼 수 있습니다.

```Shell
pip install tensorflow-datasets -U
```
glue는 다음과 같은 구조로 돼있습니다.
```python
FeaturesDict({
    'hypothesis': Text(shape=(), dtype=tf.string),
    'idx': tf.int32,
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
    'premise': Text(shape=(), dtype=tf.string),
})
```

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
print(tfds.__version__)

4.4.0


In [3]:
ds=tfds.load('glue/mnli')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/5 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/392702 [00:00<?, ? examples/s]

...

Generating test_matched examples...:   0%|          | 0/9796 [00:00<?, ? examples/s]

Shuffling glue-test_matched.tfrecord...:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched examples...:   0%|          | 0/9847 [00:00<?, ? examples/s]

Shuffling glue-test_mismatched.tfrecord...:   0%|          | 0/9847 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /aiffel/tensorflow_datasets/glue/mnli/2.0.0. Subsequent calls will reuse this data.


In [4]:
type(tfds.load('glue/mnli'))

dict

# 주어진 태스크
>STEP 1. mnli 데이터셋을 분석해 보기
tensorflow-datasets를 이용하여 glue/mnli를 다운로드하려면 tensorflow-datasets 라이브러리 버전을 올려야 합니다.

```shell
pip install tensorflow-datasets -U
```
>위 명령어를 통해 라이브러리 업그레이드를 진행해 주세요!

이건 안 해도 되는데요?

>
STEP 2. MNLIProcessor클래스 구현하기
STEP 3. 위에서 구현한 processor 및 Huggingface에서 제공하는 tokenizer를 활용하여 데이터셋 구성하기
STEP 4. model을 생성하여 학습 및 테스트를 진행해 보기
💡 힌트
혹시 STEP 2의 진행에 어려움을 겪고 계신다면 transformer 프로젝트 내부를 살펴보시면 참고할만한 예시 코드를 찾아볼 수 있을 것입니다. transformers의 공식 github을 참고하는 것도 좋은 방법이에요!

In [5]:
ds['train']

<PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

In [6]:

for idx, doc in enumerate(ds['train']):
    print(idx,doc['idx'], doc['hypothesis'], doc['label'], doc['premise'], sep='\t')
    print('\n')
    if idx==3:
        break


0	tf.Tensor(16399, shape=(), dtype=int32)	tf.Tensor(b'Meaningful partnerships with stakeholders is crucial.', shape=(), dtype=string)	tf.Tensor(1, shape=(), dtype=int64)	tf.Tensor(b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', shape=(), dtype=string)


1	tf.Tensor(206287, shape=(), dtype=int32)	tf.Tensor(b'The Clinton followers kept to the higher ground in the discussion. ', shape=(), dtype=string)	tf.Tensor(0, shape=(), dtype=int64)	tf.Tensor(b'The Clinton surrogates also held the high ground in the context war.', shape=(), dtype=string)


2	tf.Tensor(352707, shape=(), dtype=int32)	tf.Tensor(b'Women have jobs in all areas of the workforce, they are almost getting the same wages as most men,', shape=(), dtype=string)	tf.Tensor(1, shape=(

문장을 몇 개 살펴보니, label이 다음과 같음을 알 수 있습니다.   

|Label|0|1|2|
|---|---|---|---|
|관계|이어짐|무관|반례|

그런데 0, 1이 확실하지 않습니다. 잘 모르겠네요. 일단 계속 해봅시다. 

[tensorflow.org, tfds.as_dataframe](https://www.tensorflow.org/datasets/api_docs/python/tfds/as_dataframe) 을 통해 pandas 데이터프레임을 받을 수 있다고 합니다.

In [7]:
keys=list(ds.keys())
print(keys)

[Split('train'), 'validation_matched', 'validation_mismatched', 'test_matched', 'test_mismatched']


In [8]:
ds[keys[0]]

<PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

In [9]:
df=tfds.as_dataframe(ds[keys[0]])

In [10]:
df[:5]

,hypothesis,idx,label,premise
0,b'Meaningful partnerships with stakeholders is...,16399,1,"b'In recognition of these tensions, LSC has wo..."
1,b'The Clinton followers kept to the higher gro...,206287,0,b'The Clinton surrogates also held the high gr...
2,b'Women have jobs in all areas of the workforc...,352707,1,"b""um-hum because women are in every field now ..."
3,b'Houston is freezing and dry right now.',372070,2,b'Houston is really humid now'
4,"b""But they wouldn't be leaving right now. """,160184,1,b'But not now.'


In [11]:
len(df)

392702

용량이 300MB 정도라서 시간이 꽤 걸리므로 일부만 써서 테스트하겠습니다.

In [12]:
df=df[:1000]

In [13]:
df

,hypothesis,idx,label,premise
0,b'Meaningful partnerships with stakeholders is...,16399,1,"b'In recognition of these tensions, LSC has wo..."
1,b'The Clinton followers kept to the higher gro...,206287,0,b'The Clinton surrogates also held the high gr...
2,b'Women have jobs in all areas of the workforc...,352707,1,"b""um-hum because women are in every field now ..."
3,b'Houston is freezing and dry right now.',372070,2,b'Houston is really humid now'
4,"b""But they wouldn't be leaving right now. """,160184,1,b'But not now.'
...,...,...,...,...
995,b'John made me light the gas.',88879,2,"b'John strode across the room, and lit the gas. '"
996,"b""Music doesn't identify good or bad things. """,257511,2,"b""yeah i'm kind of thinking that's maybe our g..."
997,b'Lawrence Cavendish woke me up during the nig...,262058,0,b'It seemed to be the middle of the night when...
998,b'This was from another student at state.',201141,0,b'well the interesting thing was this was from...


In [14]:
data, info = tfds.load('glue/mnli', with_info=True)

In [15]:
data

{'train': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'validation_matched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'validation_mismatched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'test_matched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'test_mismatched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>}

In [16]:
import datasets
from datasets import load_dataset

huggingface_mnli_dataset = load_dataset('glue', 'mnli')
print(huggingface_mnli_dataset)

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /aiffel/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})


In [17]:
type(huggingface_mnli_dataset), huggingface_mnli_dataset

(datasets.dataset_dict.DatasetDict,
 DatasetDict({
     train: Dataset({
         features: ['premise', 'hypothesis', 'label', 'idx'],
         num_rows: 392702
     })
     validation_matched: Dataset({
         features: ['premise', 'hypothesis', 'label', 'idx'],
         num_rows: 9815
     })
     validation_mismatched: Dataset({
         features: ['premise', 'hypothesis', 'label', 'idx'],
         num_rows: 9832
     })
     test_matched: Dataset({
         features: ['premise', 'hypothesis', 'label', 'idx'],
         num_rows: 9796
     })
     test_mismatched: Dataset({
         features: ['premise', 'hypothesis', 'label', 'idx'],
         num_rows: 9847
     })
 }))

In [18]:
huggingface_mnli_dataset['train'][:5]

{'premise': ['Conceptually cream skimming has two basic dimensions - product and geography.',
  'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him',
  'One of our number will carry out your instructions minutely.',
  'How do you know? All this is their information again.',
  "yeah i tell you what though if you go price some of those tennis shoes i can see why now you know they're getting up in the hundred dollar range"],
 'hypothesis': ['Product and geography are what make cream skimming work. ',
  'You lose the things to the following level if the people recall.',
  'A member of my team will execute your orders with immense precision.',
  'This information belongs to them.',
  'The tennis shoes have a range of prices.'],
 'label': [1, 0, 0, 0, 1],
 'idx': [0,

# processor 만들기
노드를 따라해봅니다. 

In [36]:
import os
import numpy as np
from argparse import ArgumentParser
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification, AutoConfig
from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures

print("슝~")

슝~


In [20]:
class DataProcessor:
    """Base class for data converters for sequence classification data sets."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """
        Gets an example from a dict with tensorflow tensors.

        Args:
            tensor_dict: Keys and values should match the corresponding Glue
                tensorflow_dataset examples.
        """
        raise NotImplementedError()

    def get_train_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    def tfds_map(self, example):
        """
        Some tensorflow_datasets datasets are not formatted the same way the GLUE datasets are. This method converts
        examples to the correct format.
        """
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

print("슝~")

슝~


In [28]:
class MnliProcessor(DataProcessor):
    """Processor for the MNLI data set (GLUE version)."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            tensor_dict["premise"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["0", "1", "2"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            text_b = line[4]
            label = None if set_type == "test" else line[0]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples

print("슝~")

슝~


여전히 class DataProcessor의 역할을 모르겠습니다. 다음에는 저걸 실행 안하고 해봐야겠네요.  
get_example_from_tensor_dict()를 통해 간단히 프로세서의 역할을 알아봅니다.

In [29]:
processor = MnliProcessor()
examples = data['train'].take(5)

for example in examples:
    print('------원본데이터------')
    print(example)  
    example = processor.get_example_from_tensor_dict(example)
    print('------processor 가공데이터------')
    print(example)

------원본데이터------
{'hypothesis': <tf.Tensor: shape=(), dtype=string, numpy=b'Meaningful partnerships with stakeholders is crucial.'>, 'idx': <tf.Tensor: shape=(), dtype=int32, numpy=16399>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'premise': <tf.Tensor: shape=(), dtype=string, numpy=b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'>}
------processor 가공데이터------
InputExample(guid=16399, text_a='Meaningful partnerships with stakeholders is crucial.', text_b='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal

In [30]:
examples = (data['train'].take(1))
for example in examples:
    example = processor.get_example_from_tensor_dict(example)
    example = processor.tfds_map(example)
    print(example)

InputExample(guid=16399, text_a='Meaningful partnerships with stakeholders is crucial.', text_b='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', label='1')


In [31]:
label_list = processor.get_labels()
label_list

['0', '1', '2']

# 토크나이저, 모델
어떤 모델을 쓸 수 있을까요?
[huggingface.co 의 documentation](https://huggingface.co/docs/transformers/autoclass_tutorial)을 참고해서, distilbart 를 써보려고 했습니다만 일단 노드 코드를 붙여넣고 문제를 해결하다가 DataProcessor가 Deprecated Processor라고 써있는 것을 보고 포기하기로 하였습니다...

In [39]:
from transformers import BartTokenizer, BartForSequenceClassification

tokenizer = BartTokenizer.from_pretrained('valhalla/distilbart-mnli-12-9')
model = BartForSequenceClassification.from_pretrained('valhalla/distilbart-mnli-12-9')

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

In [41]:
def _glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="claasification") :
    if max_length is None :
        max_length = tokenizer.max_len
    if label_list is None:
        label_list = processor.get_labels()
        print("Using label list %s" % (label_list))

    label_map = {label: i for i, label in enumerate(label_list)}
    labels = [label_map[example.label] for example in examples]

    batch_encoding = tokenizer(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        print("*** Example ***")
        print("guid: %s" % (example.guid))
        print("features: %s" % features[i])

    return features

print("슝~")

슝~


In [42]:
def tf_glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="classification") :
    """
    :param examples: tf.data.Dataset
    :param tokenizer: pretrained tokenizer
    :param max_length: example의 최대 길이(기본값 : tokenizer의 max_len)
    :param task: GLUE task 이름
    :param label_list: 라벨 리스트
    :param output_mode: "regression" or "classification"

    :return: task에 맞도록 feature가 구성된 tf.data.Dataset
    """
    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, max_length, processor)
    label_type = tf.int64

    def gen():
        for ex in features:
            d = {k: v for k, v in asdict(ex).items() if v is not None}
            label = d.pop("label")
            yield (d, label)

    input_names = ["input_ids"] + tokenizer.model_input_names

    return tf.data.Dataset.from_generator(
        gen,
        ({k: tf.int32 for k in input_names}, label_type),
        ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
    )

print("슝~")

슝~


In [43]:
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, processor=processor)

Using label list ['0', '1', '2']


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

*** Example ***
guid: 16399
features: InputFeatures(input_ids=[0, 5096, 19266, 2650, 8670, 19, 7193, 16, 4096, 4, 2, 2, 1121, 4972, 9, 209, 5734, 6, 226, 3632, 34, 1006, 28554, 187, 7969, 7, 15451, 5, 2113, 9, 5, 331, 9619, 11980, 8, 7, 5242, 6667, 8670, 19, 7193, 3448, 23, 26242, 10, 92, 39835, 38115, 227, 5, 752, 3696, 8, 11940, 9, 1030, 518, 1435, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=None, label=1)
*** E

In [44]:
examples = train_dataset.take(1)
for example in examples:
    print(example)

({'input_ids': <tf.Tensor: shape=(128,), dtype=int32, numpy=
array([    0,  5096, 19266,  2650,  8670,    19,  7193,    16,  4096,
           4,     2,     2,  1121,  4972,     9,   209,  5734,     6,
         226,  3632,    34,  1006, 28554,   187,  7969,     7, 15451,
           5,  2113,     9,     5,   331,  9619, 11980,     8,     7,
        5242,  6667,  8670,    19,  7193,  3448,    23, 26242,    10,
          92, 39835, 38115,   227,     5,   752,  3696,     8, 11940,
           9,  1030,   518,  1435,     4,     2,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,  

In [45]:
# train 데이터셋
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, processor=processor)
train_dataset_batch = train_dataset.shuffle(100).batch(16).repeat(2)

Using label list ['0', '1', '2']


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

*** Example ***
guid: 16399
features: InputFeatures(input_ids=[0, 5096, 19266, 2650, 8670, 19, 7193, 16, 4096, 4, 2, 2, 1121, 4972, 9, 209, 5734, 6, 226, 3632, 34, 1006, 28554, 187, 7969, 7, 15451, 5, 2113, 9, 5, 331, 9619, 11980, 8, 7, 5242, 6667, 8670, 19, 7193, 3448, 23, 26242, 10, 92, 39835, 38115, 227, 5, 752, 3696, 8, 11940, 9, 1030, 518, 1435, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=None, label=1)
*** E

In [46]:
# validation 데이터셋
validation_dataset = tf_glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, processor=processor)
validation_dataset_batch = validation_dataset.shuffle(100).batch(16)

KeyError: 'validation'

In [ ]:
# test 데이터셋
test_dataset = tf_glue_convert_examples_to_features(data['test'], tokenizer, max_length=128, processor=processor)
test_dataset_batch = test_dataset.shuffle(100).batch(16)

# Train, Eval

In [ ]:
num_classes = len(processor.get_labels())

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

print("슝~")

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

model.summary()

In [ ]:
# 이전 스텝에서 배치처리를 진행한 데이터셋(xxxx_dataset_batch)을 활용
model.fit(train_dataset_batch, epochs=3, steps_per_epoch=115, 
                validation_data=validation_dataset_batch)

In [ ]:
result = model.evaluate(test_dataset_batch)
print(result)

In [ ]:
import os
output_dir = os.getenv('HOME')+'/aiffel/transformers'
output_eval_file = os.path.join(output_dir, "eval_results.txt")

with open(output_eval_file, "w") as writer:
    for i, v in enumerate(result) :
        if i == 0 :
            writer.write("Loss = %f\t" %(v))
        if i == 1 :
            writer.write("Accuracy = %f\n" %(v))
print("완료=3")

#파일에 쓴 테스트 결과 확인
!cat ~/aiffel/transformers/eval_results.txt